In [93]:
"""
This script runs simulations reported in our paper Confidence Intervals for Policy Evaluation in Adaptive Experiments (https://arxiv.org/abs/1911.02768)
"""

import sys
# sys.path.insert(0, "/home/rhzhan/adaptive-confidence-intervals/")
from time import time
from sys import argv
from random import choice
import pickle
import os
import numpy as np
from adaptive_CI.experiments import run_mab_experiment
from adaptive_CI.compute import stick_breaking
from adaptive_CI.saving import *
from adaptive_CI.inference import *
from adaptive_CI.weights import *


%reload_ext autoreload
%autoreload 2

In [ ]:
# ----------------------------------------------------
num_sims = 20000

# Read DGP specification
noise_func = 'uniform'
truths = {
    'nosignal': np.array([1., 1., 1.]),
    'lowSNR': np.array([.9, 1., 1.1]),
    'highSNR': np.array([.5, 1., 1.5])
}

results_list = []
start_time = time()


# ----------------------------------------------------
# Run simulations
for s in range(num_sims):
    if (s+1) % 10 == 0:
        print(f'Running simulation {s+1}/{num_sims}')

    """ Experiment configuration """
    experiment = choice(['nosignal', 'lowSNR', 'highSNR'])
    truth = truths[experiment]
    T = choice([1000, 5000, 10000, 20000])  # number of samples
    K = len(truth)  # number of arms
    initial = 5  # initial number of samples of each arm to do pure exploration
    floor_start = 1 / K
    floor_decay = 0.9
    exploration = 'TS'
    noise_scale = 1.0

    """ Generate data """
    if noise_func == 'uniform':
        noise = np.random.uniform(-noise_scale, noise_scale, size=(T, K))
    else:
        noise = np.random.exponential(noise_scale, size=(T, K)) - noise_scale
    ys = truth + noise

    """ Run experiment """
    data = run_mab_experiment(
        ys,
        initial=initial,
        floor_start=floor_start,
        floor_decay=floor_decay,
        exploration=exploration)

    probs = data['probs']
    rewards = data['rewards']
    arms = data['arms']

    """ Compute AIPW scores """
    muhat = np.row_stack([np.zeros(K), sample_mean(rewards, arms, K)[:-1]])
    scores = aw_scores(rewards, arms, probs, muhat)

    """ Compute weights """
    # Two-point allocation rate
    twopoint_ratio = twopoint_stable_var_ratio(probs, floor_decay)
    twopoint_h2es = stick_breaking(twopoint_ratio)
    wts_twopoint = np.sqrt(np.maximum(0., twopoint_h2es * probs))

    # Other weights: lvdl(constant allocation rate), propscore and uniform
    wts_lvdl = np.sqrt(probs)
    wts_propscore = probs
    wts_uniform = np.ones_like(probs)

    """ Estimate arm values """
    # for each weighting scheme, return [estimate, S.E, bias, 95%-coverage, t-stat, mse, truth]
    stats = dict(
        uniform=aw_stats(scores, wts_uniform, truth),
        propscore=aw_stats(scores, wts_propscore, truth),
        lvdl=aw_stats(scores, wts_lvdl, truth),
        two_point=aw_stats(scores, wts_twopoint, truth),
    )

    """ Estimate contrasts """
    contrasts = dict(
        uniform=aw_contrasts(scores, wts_uniform, truth),
        propscore=aw_contrasts(scores, wts_propscore, truth),
        lvdl=aw_contrasts(scores, wts_lvdl, truth),
        two_point=aw_contrasts(scores, wts_twopoint, truth),
    )

    """ Save results """
    weights = dict(
        uniform=wts_uniform,
        propscore=wts_propscore,
        lvdl=wts_lvdl,
        two_point=wts_twopoint,
    )

    ratios = dict(
        lvdl=np.ones((T, K)) / np.arange(T, 0, -1)[:, np.newaxis],
        two_point=twopoint_ratio,
    )

    config = dict(
        T=T,
        K=K,
        noise_func=noise_func,
        noise_scale=noise_scale,
        floor_start=floor_start,
        floor_decay=floor_decay,
        initial=initial,
        truth=truth,
    )

    # only save at saved_timepoints for assigmnment probabilities, conditional variance, weights, ratios(lambdas)
    saved_timepoints = list(range(0, T, T // 100))
    condVars = dict()
    for method, weight in weights.items():
        condVar = weight ** 2 / probs / np.sum(weight, 0) ** 2 * T
        weight = weight / np.sum(weight, 0) * T
        condVars[method] = condVar[saved_timepoints, :]
        weights[method] = weight[saved_timepoints, :]
    for ratio in ratios:
        ratios[ratio] = ratios[ratio][saved_timepoints, :]
    probs = probs[saved_timepoints, :]
    results = dict(
        config=config,
        probs=probs,
        stats=stats,
        contrasts=contrasts,
        weights=weights,
        ratios=ratios,
        condVars=condVars)

    results_list.append(results)

    write_dir = os.path.join(os.getcwd(), 'results')
    if not os.path.exists(write_dir):
        os.makedirs(write_dir)
    filename = compose_filename(f'weight_experiment_{experiment}_{noise_func}', 'pkl')
    write_path = os.path.join(write_dir, filename)
    with open(write_path, "wb") as f:
        pickle.dump(results_list, f)
    # print(filename)

    results_list = []

print(f"Time passed {time()-start_time}s")


Running simulation 10/20000
Running simulation 20/20000
Running simulation 30/20000
Running simulation 40/20000
Running simulation 50/20000
Running simulation 60/20000
Running simulation 70/20000
Running simulation 80/20000
Running simulation 90/20000
Running simulation 100/20000
Running simulation 110/20000
Running simulation 120/20000
Running simulation 130/20000
Running simulation 140/20000
Running simulation 150/20000
Running simulation 160/20000
Running simulation 170/20000
Running simulation 180/20000
Running simulation 190/20000
Running simulation 200/20000
Running simulation 210/20000
Running simulation 220/20000
Running simulation 230/20000
Running simulation 240/20000
Running simulation 250/20000
Running simulation 260/20000
Running simulation 270/20000
Running simulation 280/20000
Running simulation 290/20000
Running simulation 300/20000
Running simulation 310/20000
Running simulation 320/20000
Running simulation 330/20000
Running simulation 340/20000
Running simulation 350/